# Imports

In [1]:
import yfinance as yf
import requests
import bs4
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import datetime
import glob
import os

# Indices

## Get Tickers

In [26]:
indices = pd.read_html("https://finance.yahoo.com/world-indices/")[0]

In [27]:
indices = indices[["Symbol", "Name"]]
indices

,Symbol,Name
0,^GSPC,S&P 500
1,^DJI,Dow Jones Industrial Average
2,^IXIC,NASDAQ Composite
3,^NYA,NYSE COMPOSITE (DJ)
4,^XAX,NYSE AMEX COMPOSITE INDEX
5,^BUK100P,Cboe UK 100
6,^RUT,Russell 2000
7,^VIX,Vix
8,^FTSE,FTSE 100
9,^GDAXI,DAX PERFORMANCE-INDEX


In [28]:
indian_indices = np.array([["Nifty 50", "^NSEI"],
                           ["Nifty Bank", "^NSEBANK"],
                           ["SENSEX", "^BSESN"],
                           ["India Vix", "^INDIAVIX"]])

indian_indices = pd.DataFrame(indian_indices, columns=["Name", "Symbol"])
indices = indices.append(indian_indices, ignore_index=True)
indices

/tmp/ipykernel_13208/3383162324.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indices = indices.append(indian_indices, ignore_index=True)


,Symbol,Name
0,^GSPC,S&P 500
1,^DJI,Dow Jones Industrial Average
2,^IXIC,NASDAQ Composite
3,^NYA,NYSE COMPOSITE (DJ)
4,^XAX,NYSE AMEX COMPOSITE INDEX
5,^BUK100P,Cboe UK 100
6,^RUT,Russell 2000
7,^VIX,Vix
8,^FTSE,FTSE 100
9,^GDAXI,DAX PERFORMANCE-INDEX


In [29]:
indices.to_csv("Data/Ticker_Data/indices_all.csv", index=False)

## Download Index Data

In [30]:
indices = pd.read_csv("Data/Ticker_Data/indices.csv")
indices.head()

,Symbol,Name
0,^GSPC,S&P 500
1,^DJI,Dow Jones Industrial Average
2,^IXIC,NASDAQ Composite
3,^BUK100P,Cboe UK 100
4,^VIX,Vix


In [32]:
symbols = indices["Symbol"].values
names = indices["Name"].values
len(symbols)

24

In [34]:
data = yf.download(list(symbols), period="max", interval="1d", group_by="ticker", threads = True, auto_adjust = True)

[*********************100%***********************]  24 of 24 completed

5 Failed downloads:
- ^INDIAVIX: 1d data not available for startTime=-2208994789 and endTime=1671771919. Only 100 years worth of day granularity data are allowed to be fetched per request.
- ^NSEI: No timezone found, symbol may be delisted
- ^AXJO: No data found for this date range, symbol may be delisted
- ^N225: No data found for this date range, symbol may be delisted
- ^KS11: No data found for this date range, symbol may be delisted


In [35]:
def clean_data(ticker):
    index = data[ticker]
    index = index.dropna()
    index = index.reset_index()
    return index

In [42]:
data_dict = (dict(zip(symbols, map(clean_data, symbols))))

### Retrying for the failed inices

In [46]:
failed_indices = []
for i in tqdm.tqdm(range(len(symbols))):
    if data_dict[symbols[i]].empty:
        failed_indices.append(symbols[i])

100%|██████████| 24/24 [00:00<00:00, 40184.95it/s]


In [58]:
data_retry = yf.download(failed_indices, period="max", interval="1d", group_by="ticker", threads = True, auto_adjust = True)

[*********************100%***********************]  5 of 5 completed

1 Failed download:
- ^INDIAVIX: 1d data not available for startTime=-2208994789 and endTime=1671772519. Only 100 years worth of day granularity data are allowed to be fetched per request.


In [60]:
def clean_data_retry(ticker):
    index = data_retry[ticker]
    index = index.dropna()
    index = index.reset_index()
    return index

In [62]:
data_dict_retry = (dict(zip(failed_indices, map(clean_data_retry, failed_indices))))

In [71]:
for key in data_dict_retry.keys():
    data_dict[key] = data_dict_retry[key]

for key in tqdm.tqdm(data_dict.keys(), desc="Saving Data"):
    file_name = "Data/Indices/" + key.replace("^", "") + ".csv"
    data_dict[key].to_csv(file_name, index=False)

Saving Data: 100%|██████████| 24/24 [00:01<00:00, 17.21it/s]


# Stocks

## Get Tickers

In [115]:
nifty100 = pd.read_csv("Data/Ticker_Data/nifty100.csv")
nifty200 = pd.read_csv("Data/Ticker_Data/nifty200.csv")
nasdaq = pd.read_csv("Data/Ticker_Data/nasdaq.csv")
nasdaq200 = pd.read_csv("Data/Ticker_Data/nasdaq200.csv")

In [116]:
nasdaq200.head()

,Symbol,Name,Country,Sector,IPO Year,Industry
0,AAPL,Apple Inc. Common Stock,United States,Technology,1980.0,Computer Manufacturing
1,MSFT,Microsoft Corporation Common Stock,United States,Technology,1986.0,Computer Software: Prepackaged Software
2,GOOG,Alphabet Inc. Class C Capital Stock,United States,Technology,2004.0,Internet and Information Services
3,GOOGL,Alphabet Inc. Class A Common Stock,United States,Technology,2004.0,Internet and Information Services
4,AMZN,Amazon.com Inc. Common Stock,United States,Consumer Discretionary,1997.0,Catalog/Specialty Distribution


In [117]:
nifty100.head()

,Symbol,Name,Industry
0,ACC,ACC Ltd.,Construction Materials
1,ADANIENT,Adani Enterprises Ltd.,Metals & Mining
2,ADANIGREEN,Adani Green Energy Ltd.,Power
3,ADANIPORTS,Adani Ports and Special Economic Zone Ltd.,Services
4,ATGL,Adani Total Gas Ltd.,Oil Gas & Consumable Fuels


## Download Index Data

### NASDAQ 200

In [118]:
indices = pd.read_csv("Data/nasdaq200.csv")
indices.head()

,Symbol,Name,Country,Sector,IPO Year,Industry
0,AAPL,Apple Inc. Common Stock,United States,Technology,1980.0,Computer Manufacturing
1,MSFT,Microsoft Corporation Common Stock,United States,Technology,1986.0,Computer Software: Prepackaged Software
2,GOOG,Alphabet Inc. Class C Capital Stock,United States,Technology,2004.0,Internet and Information Services
3,GOOGL,Alphabet Inc. Class A Common Stock,United States,Technology,2004.0,Internet and Information Services
4,AMZN,Amazon.com Inc. Common Stock,United States,Consumer Discretionary,1997.0,Catalog/Specialty Distribution


In [119]:
symbols = indices["Symbol"].values
names = indices["Name"].values
len(symbols)

200

In [120]:
data = yf.download(list(symbols), period="max", interval="1d", group_by="ticker", threads = True, auto_adjust = True)

[*********************100%***********************]  200 of 200 completed


In [121]:
def clean_data(ticker):
    index = data[ticker]
    index = index.dropna()
    index = index.reset_index()
    return index

In [122]:
data_dict = (dict(zip(symbols, map(clean_data, symbols))))

In [124]:
failed_indices = []
for i in tqdm.tqdm(range(len(symbols))):
    if data_dict[symbols[i]].empty:
        failed_indices.append(symbols[i])

100%|██████████| 200/200 [00:00<00:00, 105212.69it/s]


In [125]:
failed_indices

[]

In [126]:
if failed_indices:
    data_retry = yf.download(failed_indices, period="max", interval="1d", group_by="ticker", threads = True, auto_adjust = True)

In [127]:
def clean_data_retry(ticker):
    index = data_retry[ticker]
    index = index.dropna()
    index = index.reset_index()
    return index

In [128]:
data_dict_retry = (dict(zip(failed_indices, map(clean_data_retry, failed_indices))))

In [130]:
for key in data_dict_retry.keys():
    data_dict[key] = data_dict_retry[key]

for key in tqdm.tqdm(data_dict.keys(), desc="Saving Data"):
    file_name = "Data/nasdaq200/" + key.replace("^", "") + ".csv"
    data_dict[key].to_csv(file_name, index=False)

Saving Data: 100%|██████████| 200/200 [00:14<00:00, 13.89it/s]


In [155]:
len(glob.glob("Data/nasdaq200/*.csv"))

200

### Nifty 200

In [131]:
indices = pd.read_csv("Data/nifty200.csv")
indices.head()

,Symbol,Name,Industry
0,ABB,ABB India Ltd.,Capital Goods
1,ACC,ACC Ltd.,Construction Materials
2,AUBANK,AU Small Finance Bank Ltd.,Financial Services
3,ABBOTINDIA,Abbott India Ltd.,Healthcare
4,ADANIENT,Adani Enterprises Ltd.,Metals & Mining


In [134]:
symbols = indices["Symbol"].values
symbols = list(map(lambda x:x+".BO", symbols))
print(symbols[10])
names = indices["Name"].values
len(symbols)

ABCAPITAL.BO


200

In [ ]:
data = yf.download(list(symbols), period="max", interval="1d", group_by="ticker", threads = True, auto_adjust = True)

In [136]:
def clean_data(ticker):
    index = data[ticker]
    index = index.dropna()
    index = index.reset_index()
    return index

In [137]:
data_dict = (dict(zip(symbols, map(clean_data, symbols))))

In [138]:
len(data_dict)

200

In [139]:
failed_indices = []
for i in tqdm.tqdm(range(len(symbols))):
    if data_dict[symbols[i]].empty:
        failed_indices.append(symbols[i])

100%|██████████| 200/200 [00:00<00:00, 129115.10it/s]


In [141]:
len(failed_indices)

37

In [144]:
if failed_indices:
    data_retry = yf.download(failed_indices, period="5y", interval="1d", group_by="ticker", threads = True, auto_adjust = True)

[*********************100%***********************]  37 of 37 completed

3 Failed downloads:
- DEEPAKNTR.BO: No data found, symbol may be delisted
- MCDOWELL-N.BO: No data found, symbol may be delisted
- GUJGASLTD.BO: No data found, symbol may be delisted


In [145]:
def clean_data_retry(ticker):
    index = data_retry[ticker]
    index = index.dropna()
    index = index.reset_index()
    return index

In [146]:
data_dict_retry = (dict(zip(failed_indices, map(clean_data_retry, failed_indices))))

In [149]:
for key in data_dict_retry.keys():
    data_dict[key] = data_dict_retry[key]

for key in tqdm.tqdm(data_dict.keys(), desc="Saving Data"):
    file_name = "Data/nifty200/" + key.replace(".BO", "") + ".csv"
    data_dict[key].to_csv(file_name, index=False)

Saving Data: 100%|██████████| 200/200 [00:07<00:00, 26.15it/s]


In [158]:
final_retry = ["DEEPAKNTR.NS", "MCDOWELL-N.NS","GUJGASLTD.NS"]
data_retry = yf.download(final_retry, period="max", interval="1d", group_by="ticker", threads = True, auto_adjust = True)

[*********************100%***********************]  3 of 3 completed


In [160]:
for t in final_retry:
    index = data_retry[t]
    index = index.dropna()
    index = index.reset_index()
    file_name = "Data/nifty200/" + t.replace(".NS", "") + ".csv"
    index.to_csv(file_name, index=False)

In [161]:
len(glob.glob("Data/nifty200/*.csv"))

200

# Other

In [3]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "SPY AAPL MSFT",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "max",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # Whether to ignore timezone when aligning ticker data from 
        # different timezones. Default is True. False may be useful for 
        # minute/hourly data.
        ignore_tz = False,

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # attempt repair of missing data or currency mixups e.g. $/cents
        repair = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,
    )

[*********************100%***********************]  3 of 3 completed


In [4]:
data["AAPL"]

,Open,High,Low,Close,Volume
Date,,,,,
1980-12-12 00:00:00-05:00,0.099874,0.100308,0.099874,0.099874,469033600
1980-12-15 00:00:00-05:00,0.095098,0.095098,0.094663,0.094663,175884800
1980-12-16 00:00:00-05:00,0.088149,0.088149,0.087715,0.087715,105728000
1980-12-17 00:00:00-05:00,0.089886,0.090320,0.089886,0.089886,86441600
1980-12-18 00:00:00-05:00,0.092492,0.092927,0.092492,0.092492,73449600
...,...,...,...,...,...
2022-12-16 00:00:00-05:00,136.690002,137.649994,133.729996,134.509995,160080100
2022-12-19 00:00:00-05:00,135.110001,135.199997,131.320007,132.369995,79592600
2022-12-20 00:00:00-05:00,131.389999,133.250000,129.889999,132.300003,77432800


In [6]:
apple = data["AAPL"]
apple = apple.dropna()
apple = apple.reset_index()
apple.to_csv("Data/Stocks/nasdaq200/AAPL.csv", index=False)

In [7]:
aapl = yf.Ticker("AAPL")

In [14]:
nifty = yf.download("^NSEI", period="ytd", interval="1d")

[*********************100%***********************]  1 of 1 completed


In [15]:
nifty

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,17387.150391,17646.650391,17383.300781,17625.699219,17625.699219,200500
2022-01-04,17681.400391,17827.599609,17593.550781,17805.250000,17805.250000,247400
2022-01-05,17820.099609,17944.699219,17748.849609,17925.250000,17925.250000,251500
2022-01-06,17768.500000,17797.949219,17655.550781,17745.900391,17745.900391,236500
2022-01-07,17797.599609,17905.000000,17704.550781,17812.699219,17812.699219,239300
...,...,...,...,...,...,...
2022-12-19,18288.099609,18431.650391,18244.550781,18420.449219,18420.449219,154800
2022-12-20,18340.300781,18404.900391,18202.650391,18385.300781,18385.300781,169000
2022-12-21,18435.150391,18473.349609,18162.750000,18199.099609,18199.099609,187600
